# Relative Entropy Demo

---

Authors: Aaron Spring

This is a work in progress for the relative entropy method.

In [ ]:
import xarray as xr
import numpy as np
import warnings
import os
warnings.filterwarnings("ignore")

In [ ]:
if os.getcwd().startswith('/home/travis/'):
    mistral = False
elif (os.getcwd().startswith('/mnt/')) or (os.getcwd().startswith('/Users/aaron.spring')):
    mistral = True
else:
    mistral = False
print('on mistral:',mistral,' - should be False to test with synthetic data')

if mistral:
    from PMMPIESM.setup import _get_path
    ds = xr.open_dataset(_get_path('tos',prefix='ds')).rename({'ensemble':'init','time':'lead'})
    control = xr.open_dataset(_get_path('tos',prefix='control'))
else:
    dates = np.arange(3000, 3100)
    lats = np.arange(4)
    lons = np.arange(3)
    data = np.random.rand(len(dates), len(lats), len(lons))
    control = xr.DataArray(data,
                        coords=[dates, lats, lons],
                        dims=['time', 'lat', 'lon'])
    
    lead = np.arange(1, 4)
    lats = np.arange(4)
    lons = np.arange(3)
    member = np.arange(10)
    init = [3004, 3009, 3015, 3023]
    data = np.random.rand(len(lead), len(member), len(init), len(
        lats), len(lons))
    ds = xr.DataArray(data,
                        coords=[lead, member, init, lats, lons],
                        dims=['lead', 'member', 'init', 'lat', 'lon'])

In [ ]:
from climpred.relative_entropy import compute_relative_entropy, bootstrap_relative_entropy
from climpred.graphics import plot_relative_entropy

## perfect-model

In [ ]:
rel_ent = compute_relative_entropy(ds, control,nmember_control=100,neofs=8)

In [ ]:
rel_ent.to_dataframe().unstack(0).head()

In [ ]:
rel_ent_threshold = bootstrap_relative_entropy(ds, control, sig=50,
                               bootstrap=50, curv=True, neofs=8,
                               nmember_control=100)

In [ ]:
plot_relative_entropy(rel_ent, rel_ent_threshold=rel_ent_threshold, sharey=True, figsize=(15,4))

## DPLE, LENS
faking data with DPLE, LENS coords

In [ ]:
def _shuffle(ds, dim='init'):
    """Shuffle ensemble members to uninitialize the data."""
    old_dim_range = ds[dim]
    shuffled = ds.sel({dim: np.random.permutation(ds[dim])})
    shuffled[dim] = old_dim_range
    shuffled = shuffled.sortby(dim)
    return shuffled

In [ ]:
lens = xr.concat([_shuffle(control,'time') for _ in range(10)],dim='member').isel(time=slice(0,65))
lens['time']=np.arange(1950,2015)
lens['member']=np.arange(lens.member.size)
#lens

In [ ]:
dple = xr.concat([lens.rename({'time':'init'})]*10,'lead')
dple['lead']=np.arange(1,1+dple.lead.size)
#dple

In [ ]:
compute_relative_entropy(dple, lens,nlead=4,nmember_control=5,neofs=3).to_dataframe().unstack(0).head()

In [ ]:
bootstrap_relative_entropy(dple, lens,nlead=4,nmember_control=5,neofs=3,bootstrap=15)

## Understand nmember_control

In [ ]:
import matplotlib.pyplot as plt
neofs=8
for i in [5,10,20,50,100]:
    print('\n nmember_control =',i)
    rel_ent = compute_relative_entropy(ds, control,nmember_control=i,neofs=neofs)
    el_ent_threshold = bootstrap_relative_entropy(ds, control, sig=50,
                               bootstrap=50, curv=True, neofs=neofs,
                               nmember_control=i)
    plot_relative_entropy(rel_ent, rel_ent_threshold=rel_ent_threshold, sharey=True, figsize=(15,4))
    plt.show()